In [42]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegressionCV

featurized_data_file = 'IMDB_sentiment.parquet' # 'sentiment_data_categories_featurized.parquet'

In [79]:
text_df = pd.read_parquet(featurized_data_file)
text_df.columns = ['text', 'label', 'vector']
text_df['binary_label'] = [ 1 if x=='positive' else 0 for x in text_df['label']]
text_df.head()

,text,label,vector,binary_label
0,One of the other reviewers has mentioned that ...,positive,"[0.030099309980869293, 0.050417669117450714, -...",1
1,A wonderful little production. <br /><br />The...,positive,"[-0.012201860547065735, 0.05196147784590721, -...",1
2,I thought this was a wonderful way to spend ti...,positive,"[0.014258158393204212, -0.0791383758187294, 0....",1
3,Basically there's a family where a little boy ...,negative,"[-0.041720371693372726, 0.010464085265994072, ...",0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[-0.03167529031634331, 0.00642420444637537, -0...",1


In [153]:
sentiment_clf = LogisticRegressionCV(cv=5, scoring='roc_auc',
                                     n_jobs=-1, max_iter=10000)
X_emb = [v for v in text_df['vector']]
y = text_df['binary_label']
sentiment_clf.fit(X_emb, y)

get_mean_xval_score_for_binary_classifier = lambda clf: np.mean([np.max(v) for v in clf.scores_[clf.classes_[1]]])
get_mean_xval_score_for_binary_classifier(sentiment_clf)  # 0.8182599999999999

0.9003717999999999

In [154]:
y_hat0 = sentiment_clf.predict_proba(X_emb)[:,1]

fpr, tpr, thresholds = metrics.roc_curve(y, y_hat0)
metrics.auc(fpr, tpr)

0.9046707968000001

In [81]:
# Cluster the cases by weighted semantic embedding and predict the clusters

to_unit_vector = lambda v: np.array(v/np.sqrt(np.sum(v*v)))

unit_coef_vector = to_unit_vector(sentiment_clf.coef_)[0]  # ???

X_weighted = np.array([ unit_coef_vector * v for v in text_df['vector']])


In [88]:
text_df['score2'] = [np.sum(v) for v in X_weighted]

import numpy as np
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(text_df['binary_label'], text_df['score2'])
metrics.auc(fpr, tpr)

0.9046707968000001

In [89]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(X_weighted)
# Counter(kmeans.labels_)

text_df['cluster'] = kmeans.labels_
text_df.groupby('cluster').agg({'binary_label': 'mean'})

,binary_label
cluster,
0,0.600041
1,0.378049
2,0.269825
3,0.552006
4,0.751390


Now make 2k separate categories for the positive and negative cases within each cluster, and predict those with a multinomial classifier.

In [95]:
text_df['ccat_label'] = ['c' + str(row['cluster']) + 's' + str(row['binary_label']) for row in text_df.to_dict(orient='records')]

text_df

,text,label,vector,binary_label,cluster,score2,ccat_label
0,One of the other reviewers has mentioned that ...,positive,"[0.030099309980869293, 0.050417669117450714, -...",1,0,-0.008220,c0s1
1,A wonderful little production. <br /><br />The...,positive,"[-0.012201860547065735, 0.05196147784590721, -...",1,4,0.025556,c4s1
2,I thought this was a wonderful way to spend ti...,positive,"[0.014258158393204212, -0.0791383758187294, 0....",1,3,-0.017175,c3s1
3,Basically there's a family where a little boy ...,negative,"[-0.041720371693372726, 0.010464085265994072, ...",0,1,-0.068911,c1s0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[-0.03167529031634331, 0.00642420444637537, -0...",1,0,0.087525,c0s1
...,...,...,...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,"[-0.031536709517240524, -0.06321507692337036, ...",1,2,0.028319,c2s1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"[-0.04839408025145531, -0.08552412688732147, 0...",0,4,-0.132043,c4s0
49997,I am a Catholic taught in parochial elementary...,negative,"[-0.03935423120856285, -0.002203170442953706, ...",0,4,-0.066901,c4s0
49998,I'm going to have to disagree with the previou...,negative,"[-0.01833348162472248, -0.026902485638856888, ...",0,0,-0.040115,c0s0


In [103]:
# category_clf = LogisticRegressionCV(cv=5, scoring='roc_auc', n_jobs=-1, max_iter=10000, multi_class='ovr', solver='lbfgs') #multinomial , solver='lbfgs'
# category_clf.fit(X_emb, text_df['ccat_label'])

# category_clf.coef_.shape # (10, 384)

array([[ 1.46758681, -0.3262191 , -0.54736983, ..., -0.78718889,
        -0.8240993 , -2.30844094],
       [ 0.7263078 ,  0.48106036, -4.93787748, ..., -1.15162691,
         0.16843801,  2.4044313 ],
       [ 2.55087747,  0.47964774, -0.48959666, ...,  0.75008499,
        -0.16324275, -0.81952582],
       ...,
       [-1.70562752, -0.46141821, -0.78334267, ...,  0.1632658 ,
        -0.4218756 , -0.3679713 ],
       [ 3.02415178, -1.28114591, -1.80577395, ..., -0.41676819,
        -0.7781784 , -0.38803837],
       [-2.71508513, -0.55943649, -3.08604026, ...,  0.89775432,
        -1.9236693 , -0.37443604]])

In [107]:
category_clf = LogisticRegressionCV(cv=5, 
                                    # scoring='roc_auc',  # not for multinomial
                                    multi_class='multinomial', 
                                    # solver='lbfgs',
                                    n_jobs=-1, max_iter=10000) 

category_clf.fit(X_emb, text_df['ccat_label']) # category_clf.coef_.shape # (10, 384)

pprob = category_clf.predict_proba(X_emb) # pprob.shape # (50000, 10)
pos_idx = np.where([int(cc[-1]) for cc in category_clf.classes_])
best_pos_score = [np.max(v) for v in pprob[:, pos_idx][:,0,:] ] # where did the middle dimension come from?

(10, 384)

In [113]:
pa_df = pd.DataFrame({'actual': text_df['ccat_label'],
                      'predicted': category_clf.predict(X_emb)})

pa_df['is_correct'] = [row['actual'] == row['predicted'] for row in pa_df.to_dict(orient='records')]

ctr = Counter(pa_df['is_correct'])
accuracy = ctr[True]/len(pa_df)
accuracy

# LogisticRegressionSubCat Class Implementation

In [156]:
# For more detailed implementation, see https://medium.com/@hardik.prabhu/object-oriented-programming-and-ml-model-development-in-python-ada4bf76529b

import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegressionCV
from sklearn.cluster import KMeans

class LogisticRegressionSubCat():
    """ 
    A SubCategory ensemble classifier model based on LogisticRegressionCV.
    """
    
    def __init__(self, k=5):
        self.k = k
        self.clf0 = LogisticRegressionCV(cv=5, n_jobs=-1, max_iter=10000)
        self.kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto")
        self.category_clf = LogisticRegressionCV(cv=5, multi_class='multinomial', n_jobs=-1, max_iter=10000) 
    
    
    def fit(self, X, y):
        """
        X is a 2D numpy array
        """
        self.clf0.fit(X, y)
        to_unit_vector = lambda v: np.array(v/np.sqrt(np.sum(v*v)))
        unit_coef_vector = to_unit_vector(self.clf0.coef_)[0] # ???
        X_weighted = np.array([ unit_coef_vector * v for v in X])
        self.kmeans.fit(X_weighted)
        self.ccat_label = [f"c{clust:02d}_{lbl}" for clust, lbl in zip(self.kmeans.labels_, y)]

        self.category_clf.fit(X, self.ccat_label)

    
    def predict_proba(self, X):
        """
        Returns the best score for any cluster.
        """
        pprob = category_clf.predict_proba(X)
        pos_idx = np.where([int(cc[-1]) for cc in self.category_clf.classes_])
        best_pos_score = [np.max(v) for v in pprob[:, pos_idx][:,0,:] ] # !!!??? where did the middle dimension come from?
        return best_pos_score


In [169]:
X_emb = [v for v in text_df['vector']]
y = text_df['binary_label']

lrsc = LogisticRegressionSubCat(k=6)
lrsc.fit(X_emb, y)
y_hat = lrsc.predict_proba(X_emb) # !!! This is just a test; now try it on a held-out test set

fpr, tpr, thresholds = metrics.roc_curve(y, y_hat)
metrics.auc(fpr, tpr) # k=1: 0.6261, k=2: 0.5645, k=3:0.5626, k=4: 0.6513, k=5:0.9158, k=6, 8 or 10 fails (index out of bounds)

IndexError: index 11 is out of bounds for axis 1 with size 10

In [167]:
lrsc.ccat_label

['c04_1',
 'c04_1',
 'c03_1',
 'c02_0',
 'c04_1',
 'c00_1',
 'c03_1',
 'c03_0',
 'c05_0',
 'c00_1',
 'c00_0',
 'c02_0',
 'c04_0',
 'c05_0',
 'c00_1',
 'c00_0',
 'c05_1',
 'c02_0',
 'c00_1',
 'c02_0',
 'c05_1',
 'c02_0',
 'c01_1',
 'c05_0',
 'c05_0',
 'c04_1',
 'c02_1',
 'c01_0',
 'c05_0',
 'c01_1',
 'c03_1',
 'c01_1',
 'c00_0',
 'c01_1',
 'c02_0',
 'c02_0',
 'c04_0',
 'c02_0',
 'c03_1',
 'c02_0',
 'c05_0',
 'c00_1',
 'c02_0',
 'c00_0',
 'c00_1',
 'c05_1',
 'c04_0',
 'c02_0',
 'c03_1',
 'c04_0',
 'c01_1',
 'c04_1',
 'c04_1',
 'c05_1',
 'c01_0',
 'c02_0',
 'c05_0',
 'c00_0',
 'c02_1',
 'c04_1',
 'c04_0',
 'c04_0',
 'c00_1',
 'c05_0',
 'c04_0',
 'c02_1',
 'c04_0',
 'c01_0',
 'c05_0',
 'c02_0',
 'c02_0',
 'c02_0',
 'c01_1',
 'c00_1',
 'c02_0',
 'c05_1',
 'c04_1',
 'c02_0',
 'c01_0',
 'c00_1',
 'c01_1',
 'c04_0',
 'c02_0',
 'c05_0',
 'c02_0',
 'c05_0',
 'c04_0',
 'c02_0',
 'c04_0',
 'c05_0',
 'c02_1',
 'c02_0',
 'c04_1',
 'c00_1',
 'c05_0',
 'c04_1',
 'c01_0',
 'c05_0',
 'c02_0',
 'c03_1',
